# 🚀 Fine-Tuning QLoRA - Google Colab

Este notebook implementa fine-tuning de LLMs usando QLoRA (Quantized LoRA).

## 📋 Pré-requisitos

1. ✅ Google Colab com **GPU T4** (gratuito)
2. ✅ Dataset preparado (use `prepare_finetuning_data.ipynb`)
3. ✅ Conta HuggingFace (para baixar modelos)

## 🎯 O que este notebook faz:

1. Configura ambiente com GPU
2. Instala dependências QLoRA
3. Carrega dataset do Google Drive
4. Carrega modelo base com quantização 4-bit
5. Aplica LoRA adapters
6. Fine-tuna com seu dataset
7. Salva modelo treinado
8. Exporta para usar localmente

## ⏱️ Tempo estimado: 30 min - 2 horas

Depende do tamanho do modelo e dataset.

## 💡 Dica

Execute célula por célula e monitore uso de GPU/RAM.

---

**⚠️ IMPORTANTE: Ative GPU no Colab**

`Runtime > Change runtime type > Hardware accelerator > T4 GPU`


## 🔍 Verificar GPU


In [ ]:
!nvidia-smi

import torch
print(f"\n🔥 PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


## 📦 Instalar Dependências


In [ ]:
!pip install -q -U transformers accelerate peft bitsandbytes datasets trl
!pip install -q -U wandb tensorboard  # Opcional: para tracking

print("✅ Dependências instaladas!")


## 🔐 Autenticação HuggingFace

Para baixar modelos privados (como Llama 2).


In [ ]:
from huggingface_hub import login

# Cole seu token HuggingFace aqui (https://huggingface.co/settings/tokens)
HF_TOKEN = ""  # Ou use: !huggingface-cli login

if HF_TOKEN:
    login(token=HF_TOKEN)
    print("✅ Autenticado no HuggingFace!")
else:
    print("⚠️ Token não fornecido - apenas modelos públicos disponíveis")


## 📂 Montar Google Drive

Para acessar seu dataset.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive montado!")


## ⚙️ Configuração do Treinamento


In [ ]:
# ========================================
# CONFIGURAÇÕES - AJUSTE AQUI
# ========================================

# Modelo base (escolha um)
MODEL_NAME = "microsoft/phi-2"  # 2.7B - rápido, bom para testes
# MODEL_NAME = "meta-llama/Llama-2-7b-hf"  # 7B - melhor qualidade, mais lento
# MODEL_NAME = "mistralai/Mistral-7B-v0.1"  # 7B - excelente qualidade

# Caminho do dataset no Google Drive
DATASET_PATH = "/content/drive/MyDrive/training_dataset"  # Ajuste!

# Nome do modelo final
OUTPUT_MODEL_NAME = "phi2-retail-media"

# Diretório de saída
OUTPUT_DIR = "./results"

# Parâmetros de treinamento
EPOCHS = 3
BATCH_SIZE = 4  # Reduza se ficar sem memória
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 2048

# LoRA config
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# Logging
USE_WANDB = False  # True para usar Weights & Biases
WANDB_PROJECT = "qlora-finetuning"

print("✅ Configuração definida!")
print(f"📦 Modelo: {MODEL_NAME}")
print(f"📁 Dataset: {DATASET_PATH}")
print(f"🎯 Output: {OUTPUT_MODEL_NAME}")


## 📊 Carregar Dataset


In [ ]:
from datasets import load_from_disk, load_dataset

# Carregar dataset do Google Drive
try:
    dataset = load_from_disk(DATASET_PATH)
    print(f"✅ Dataset carregado do Google Drive!")
except:
    print("⚠️ Não foi possível carregar do Drive, usando dataset de exemplo...")
    # Criar dataset de exemplo para testes
    from datasets import Dataset
    
    sample_data = [
        {"text": "### Instruction:\nO que é Retail Media?\n\n### Response:\nRetail Media é uma forma de publicidade digital onde varejistas monetizam seus sites..."},
        {"text": "### Instruction:\nExplique RTB\n\n### Response:\nRTB (Real-Time Bidding) é um processo de compra automatizada de anúncios..."},
        # Adicione mais exemplos ou carregue de outro lugar
    ]
    
    dataset_dict = {"train": Dataset.from_list(sample_data), "test": Dataset.from_list(sample_data[:1])}
    from datasets import DatasetDict
    dataset = DatasetDict(dataset_dict)

print(f"\n📊 Dataset info:")
print(f"   Train: {len(dataset['train'])} exemplos")
if 'test' in dataset:
    print(f"   Test: {len(dataset['test'])} exemplos")

print(f"\n📝 Exemplo:")
print(dataset['train'][0]['text'][:200] + "...")


## 🤖 Carregar Modelo com Quantização 4-bit


In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("🔄 Carregando modelo e tokenizer...")

# Configurar quantização 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Carregar modelo
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_cache=False,  # Necessário para gradient checkpointing
)

# Preparar para treinamento k-bit
model = prepare_model_for_kbit_training(model)

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Configurar padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("✅ Modelo e tokenizer carregados!")
print(f"📊 Modelo: {MODEL_NAME}")
print(f"💾 Tamanho: {model.get_memory_footprint() / 1024**3:.2f} GB")


## 🎯 Aplicar LoRA


In [ ]:
# Configurar LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Ajuste para seu modelo
    bias="none",
    task_type="CAUSAL_LM",
)

# Aplicar LoRA ao modelo
model = get_peft_model(model, lora_config)

# Mostrar parâmetros treináveis
model.print_trainable_parameters()

print("\n✅ LoRA aplicado!")
print(f"🎯 Rank: {LORA_R}")
print(f"📊 Alpha: {LORA_ALPHA}")


## 🔤 Tokenizar Dataset


In [ ]:
def tokenize_function(examples):
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding="max_length",
        return_tensors=None,
    )
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

print("🔄 Tokenizando dataset...")

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing",
)

print("✅ Dataset tokenizado!")
print(f"   Train: {len(tokenized_dataset['train'])} exemplos")
if 'test' in tokenized_dataset:
    print(f"   Test: {len(tokenized_dataset['test'])} exemplos")


## 🚀 Configurar e Iniciar Treinamento


In [ ]:
# Configurar argumentos de treinamento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit",
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    evaluation_strategy="steps" if 'test' in tokenized_dataset else "no",
    eval_steps=100 if 'test' in tokenized_dataset else None,
    save_total_limit=3,
    seed=42,
    report_to="wandb" if USE_WANDB else "none",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Criar trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset.get("test"),
    data_collator=data_collator,
)

print("✅ Trainer configurado!")
print(f"\n📊 Parâmetros de treinamento:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"   Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Max sequence length: {MAX_SEQ_LENGTH}")

print("\n🚀 Iniciando treinamento...")
print("="*80)


In [ ]:
# TREINAR!
trainer.train()

print("\n" + "="*80)
print("✅ TREINAMENTO CONCLUÍDO!")
print("="*80)


## 💾 Salvar Modelo


In [ ]:
import os

# Criar diretório de saída
final_output_dir = f"./models/{OUTPUT_MODEL_NAME}"
os.makedirs(final_output_dir, exist_ok=True)

# Salvar apenas adaptadores LoRA (muito menor!)
model.save_pretrained(final_output_dir)
tokenizer.save_pretrained(final_output_dir)

print(f"✅ Modelo salvo em: {final_output_dir}")
print(f"📦 Tamanho dos adaptadores: ~{sum(os.path.getsize(os.path.join(dirpath,filename)) for dirpath, dirnames, filenames in os.walk(final_output_dir) for filename in filenames) / 1024 / 1024:.1f} MB")

# Copiar para Google Drive (backup)
drive_output = f"/content/drive/MyDrive/finetuned_models/{OUTPUT_MODEL_NAME}"
!mkdir -p "{drive_output}"
!cp -r "{final_output_dir}"/* "{drive_output}/"

print(f"\n✅ Backup salvo no Google Drive: {drive_output}")


## 🧪 Testar Modelo


In [ ]:
def generate_response(prompt, max_length=512):
    """Gera resposta usando o modelo fine-tunado."""
    
    # Formatar prompt (Alpaca format)
    formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"
    
    # Tokenizar
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    # Gerar
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    # Decodificar
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extrair apenas a resposta
    if "### Response:" in response:
        response = response.split("### Response:")[1].strip()
    
    return response

# Testar
test_prompts = [
    "O que é Retail Media?",
    "Explique o conceito de RTB",
    "Como funciona programmatic advertising?",
]

print("🧪 Testando modelo fine-tunado:")
print("="*80)

for prompt in test_prompts:
    print(f"\n📝 Pergunta: {prompt}")
    print(f"💬 Resposta: {generate_response(prompt)}")
    print("-"*80)


## 📦 Comprimir para Download


In [ ]:
# Comprimir modelo para facilitar download
!cd models && zip -r {OUTPUT_MODEL_NAME}.zip {OUTPUT_MODEL_NAME}

print(f"✅ Modelo comprimido: models/{OUTPUT_MODEL_NAME}.zip")
print(f"\n📥 Para usar localmente:")
print(f"   1. Baixe o arquivo ZIP")
print(f"   2. Descompacte na sua máquina")
print(f"   3. Use o código em export_to_ollama.py para converter para Ollama")


## 📋 Resumo e Próximos Passos

### ✅ O que fizemos:

1. ✅ Configuramos ambiente com GPU
2. ✅ Instalamos dependências QLoRA
3. ✅ Carregamos modelo base com quantização 4-bit
4. ✅ Aplicamos LoRA adapters
5. ✅ Fine-tunamos com seu dataset
6. ✅ Salvamos modelo treinado
7. ✅ Testamos o modelo

### 📥 Arquivos gerados:

- **Adaptadores LoRA**: `models/{OUTPUT_MODEL_NAME}/` (~50-200 MB)
- **Backup no Drive**: Salvo automaticamente
- **ZIP para download**: `models/{OUTPUT_MODEL_NAME}.zip`

### 🏠 Próximos passos (localmente):

1. **Download**: Baixe o modelo treinado do Colab/Drive
2. **Merge**: Use `export_to_ollama.py` para fazer merge com modelo base
3. **Converter**: Converta para GGUF usando llama.cpp (opcional)
4. **Ollama**: Crie Modelfile e use com `ollama run`

### 💡 Dicas:

- **Qualidade**: Se respostas não estão boas, treine mais epochs ou aumente dataset
- **Overfitting**: Se modelo decorou respostas, reduza epochs ou aumente dataset
- **Memória**: Se ficou sem GPU memory, reduza `BATCH_SIZE` ou `MAX_SEQ_LENGTH`
- **Velocidade**: Modelos menores (Phi-2) são mais rápidos que 7B

### 📚 Recursos:

- [QLoRA Paper](https://arxiv.org/abs/2305.14314)
- [PEFT Documentation](https://huggingface.co/docs/peft)
- [Transformers Documentation](https://huggingface.co/docs/transformers)

---

🎉 **Parabéns! Você fine-tunou seu primeiro LLM com QLoRA!**
